In [64]:
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import LabelEncoder

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D,BatchNormalization
from keras.optimizers import Adam, SGD, Adagrad, Adadelta, RMSprop
from keras.utils import to_categorical

import tensorflow as tf
import random as rn

# for manipulating zipped images and getting numpy arrays of pixel values of images
import cv2 # if doesn't work, you need to pip install opencv-python :)
import numpy as np
from tqdm import tqdm
import os
from random import shuffle
from zipfile import ZipFile
from PIL import Image

In [65]:
imgs=[]
labels=[]
df = []
df = pd.read_csv("HAM10000_metadata.csv")

lesion_type_dict = {
    "nv": "Melanocytic nevi",
    "mel": "Melanoma",
    "bkl": "Benign keratosis-like lesions",
    "akiec": "Actinic keratoses",
    "vasc": "Vascular lesions",
    "df": "Dermatofibroma",
    "bcc": "Basal Cell Carcinoma",
}

In [66]:
# features and target variables for train test split
# features = df[["dx_type", "age", "sex", "localization", "img"]]
features=df[["localization"]]
features = pd.get_dummies(features,columns=["localization"])
target = df[["dx"]]
target = pd.get_dummies(target, columns=["dx"])

In [67]:
x_train = features[:7000]
x_validate = features[7000:8000]
x_test = features[8000:]

y_train = target[:7000]
y_validate = target[7000:8000]
y_test = target[8000:]

In [68]:
# le=LabelEncoder()
# Y=le.fit_transform(features)
# Y=to_categorical(Y,2)
# df=np.array(df)


In [69]:
np.random.seed(42)
rn.seed(42)
tf.random.set_seed(42)

In [70]:
model = Sequential()
model.add(Conv2D(filters=32,kernel_size=(5,5),padding="Same",activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(3,3),padding="Same",activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=96,kernel_size=(3,3),padding="Same",activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=96,kernel_size=(3,3),padding="Same",activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(512)) 
model.add(Activation("relu"))
model.add(Dense(5,activation="softmax"))


In [71]:
# Define the optimizer
# optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, amsgrad=False)
# Compile the model
model.compile(optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0005), loss = "categorical_crossentropy", metrics=["accuracy"])
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [78]:
epochs = 10
batch_size = 100
history = model.fit(x_train,y_train, batch_size=batch_size, epochs = epochs, validation_data = (x_validate,y_validate), verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size, callbacks=[learning_rate_reduction])

Epoch 1/10


ValueError: in user code:

    File "/Users/karengao/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/Users/karengao/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/karengao/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/Users/karengao/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/Users/karengao/anaconda3/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/karengao/anaconda3/lib/python3.11/site-packages/keras/src/engine/input_spec.

In [73]:
# le=LabelEncoder()
# Y=le.fit_transform(target)
# Y=to_categorical(Y,6)


In [74]:
batch_size=128
epochs=10

from keras.callbacks import ReduceLROnPlateau
red_lr=ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.1)

In [75]:
datagen=ImageDataGenerator(
    featurewise_center=False, #set input mean to 0 over dataset
    samplewise_center=False, #set each sample mean to 0
    featurewise_std_normalization=False, #divide inputs by std of dataset
    samplewise_std_normalization=False, #divide each input by its std
    zca_whitening=False,
    rotation_range=10, #randomly rotate images in the range 0 to 180 degrees
    zoom_range=0.1, #randomly zoom image
    width_shift_range=0.2, #randomly shift images horizontally
    height_shift_range=0.2, #randomly shift images vertically
    horizontal_flip=True, #randomly flip images
    vertical_flip=False #randomly flip images
)

In [76]:
model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
#lr learning rate: more data -> use smaller number

In [77]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.